In [3]:
import pandas as pd
from pathlib import Path
import sys
import numpy as np

PATH = '/Users/danil/Documents/github/jooble'
sys.path.append(str(PATH))

In [16]:
train = pd.read_csv(PATH + '/data/raw/train.tsv', sep ='\t')

In [17]:
train.head()

,id_job,features
0,1864791934054678713,"2,9835,9999,9941,9945,9386,9899,9421,9954,9952..."
1,-7413918695841089440,"2,9082,9999,9700,9669,9981,9729,9822,9667,9526..."
2,-9223271545392256405,"2,9064,9999,9730,9585,9890,9740,9751,9538,9590..."
3,-9223240803898726824,"2,9402,9999,9711,9742,9975,9728,9924,9692,9514..."
4,-9223102057156184105,"2,9655,9996,9751,9722,9925,9736,9987,9733,9475..."


In [18]:
train.columns

Index(['id_job', 'features'], dtype='object')

In [19]:
train['id_job']

0      1864791934054678713
1     -7413918695841089440
2     -9223271545392256405
3     -9223240803898726824
4     -9223102057156184105
              ...         
794   -9168651328982232304
795   -9168503098892640218
796   -9168200676135763139
797   -9168200673481327350
798   -9168049274035838891
Name: id_job, Length: 799, dtype: int64

требования: 

чтобы читался файл на 50 mln строк - dask

чтобы удобно было добавлть новые паттерны учитывая первое значение в фиче

использование другой нормализации кроме z 

1. открываем train.tsc при помощи (pandas -> dask -> multiprocess. compare)

2. разбиваем фичи на колонки

3. треним нормализатор (сначало просто стандартный нормализатор -> с учетом набора -> делаем удобную добавку в классах)

    - сохраняем нормализатор (если нужно)
    

4. открываем test.tsc

5. разбиваем фичи на колонки

6. оцениваем max_feature_2_index

7. оцениваем max_feature_2_abs_mean_diff

8. нормализируем

9. переименовуем feature_2 на feature_2_stand

10. сохраняем test_proc.tsc c 

    - id_job, 

    - feature_2_stand_{i} double, 

    - max_feature_2_index int - индекс самой большой фичи у этой работы, 
    - max_feature_2_abs_mean_diff - абсолютное отклонение признака с индексом max_feature_2_index от его среднего значения 
mean(feature_2_{max_feature_2_index})

Double – это 64-битная точность

1. открываем train.tsc при помощи (dask -> multiprocess. compare)


3. треним нормализатор (с учетом набора -> делаем удобную добавку в классах)

    - сохраняем нормализатор (если нужно)

Double – это 64-битная точность

# open data

In [61]:
train = pd.read_csv(PATH + '/data/raw/train.tsv', sep ='\t')

# prepare features

In [266]:
def extract_features(df):
    parsed_features = df['features'].str.split(',', expand = True).astype(np.int16)
    parsed_features.insert(0, 'id_job', df['id_job'])
    return parsed_features

In [267]:
train_feautres = extract_features(train)

In [64]:
train_feautres.head()

,id_job,0,1,2,3,4,5,6,7,8,...,247,248,249,250,251,252,253,254,255,256
0,1864791934054678713,2,9835,9999,9941,9945,9386,9899,9421,9954,...,8818,9954,9925,9934,8689,9958,9086,9114,9950,9875
1,-7413918695841089440,2,9082,9999,9700,9669,9981,9729,9822,9667,...,9979,9752,9695,9676,9974,9788,9955,9907,9747,9824
2,-9223271545392256405,2,9064,9999,9730,9585,9890,9740,9751,9538,...,9930,9705,9645,9652,9954,9582,9947,9876,9722,9791
3,-9223240803898726824,2,9402,9999,9711,9742,9975,9728,9924,9692,...,9959,9737,9611,9661,9912,9768,9963,9971,9767,9809
4,-9223102057156184105,2,9655,9996,9751,9722,9925,9736,9987,9733,...,9908,8208,9631,9571,9971,9407,9964,9990,9783,9907


In [276]:
class preprocess:
    
    def __init__(self, normalization = 'z-score'):
        
        self.normalization = normalization
        
    def get_features_num(self, X):
        
        self.factor = X[0].value_counts().index[0]
        if self.factor == 2:
            self.features_num = 256
        # in case of selecting special dtype, further we can select only specific column type
            # self.dtypes = int
        # here wa can add new factor check for features selection or checks
        # or redefine normalization
            # self.normalization = 'z-score'

            
    def fit_scaler(self, X): 
        
        X = X.iloc[:, 2:2 + self.features_num] #.select_dtypes(include = self.dtypes)
        
        if self.normalization == 'z-score':
            self.mean  = X.mean(axis = 0)
            self.sigma = X.std(axis = 0)
            
        elif self.normalization == 'min-max-scaler':
            self.min = X.min(axis = 0)
            self.max = X.max(axis = 0)
            
            
    def transform_with_scaler(self, X):
        
        X_to_transform = X.iloc[:, 2:2 + self.features_num] #.select_dtypes(include = self.dtypes)
        
        if self.normalization == 'z-score':
            X_to_transform = (X_to_transform - self.mean) / self.sigma
            
        elif self.normalization == 'min-max-scaler':
            X_to_transform = (X_to_transform - self.min) / (self.max - self.min)
        
        X.iloc[:, 2:2 + self.features_num] = X_to_transform
        
        return X
    
    
    def add_max_feature_index(self, X):
        
        X['max_feature_2_index'] = X.iloc[:, 2: 2+ self.features_num].values.argmax(axis = 1)
        
        return X
    
    
    def add_max_feature_2_abs_mean_diff(self, X):
                        
        max_values = X.iloc[:, 2: 2+ self.features_num].values.max(axis = 1)
        mean_values = X.iloc[:, 2:2+ self.features_num].values.mean(axis = 0)
        
        if 'max_feature_2_index' in X.columns:
            indexes = X['max_feature_2_index']
        else:
            indexes = self.add_max_feature_index(X)['max_feature_2_index']

        X['max_feature_2_abs_mean_diff'] = np.abs(max_values - mean_values[indexes])


        return X
    
    
    


# fit z scaler

In [277]:
prep = preprocess()

prep.get_features_num(train_feautres)
prep.fit_scaler(train_feautres)

test_feautres.head()

,id_job,0,1,2,3,4,5,6,7,8,...,248,249,250,251,252,253,254,255,256,max_feature_2_index
0,-9168029089769934451,2,9954,9999,9974,9975,9588,9958,9851,9980,...,9981,9965,9975,9804,9979,9547,9794,9978,9961,161
1,-9167993139315005259,2,9254,9999,9695,9726,9947,9598,9866,9721,...,9720,9608,9668,9916,9790,9923,9914,9768,9794,161
2,-9167993136660569470,2,9251,9999,9605,9650,9896,9678,9857,9623,...,9699,9549,9527,9895,9718,9930,9845,9697,9692,161
3,-9167993126042826314,2,9148,9998,9470,9455,9935,9531,9935,9419,...,9456,9361,9372,9954,9402,9948,9926,9537,9671,161
4,-9167914043308884846,2,8973,9753,9347,9216,9783,9442,9966,9049,...,8038,9312,8937,9910,9254,9868,9936,9409,9782,203


# open test data

In [278]:
test = pd.read_csv(PATH + '/data/raw/test.tsv', sep ='\t')

In [279]:
test_feautres = extract_features(test)

# max_feature_2_index

In [280]:
test_feautres = prep.add_max_feature_index(test_feautres)

# max_feature_2_abs_mean_diff

In [281]:
test_feautres = prep.add_max_feature_2_abs_mean_diff(test_feautres)

In [282]:
test_feautres.head()

,id_job,0,1,2,3,4,5,6,7,8,...,249,250,251,252,253,254,255,256,max_feature_2_index,max_feature_2_abs_mean_diff
0,-9168029089769934451,2,9954,9999,9974,9975,9588,9958,9851,9980,...,9965,9975,9804,9979,9547,9794,9978,9961,161,154.17
1,-9167993139315005259,2,9254,9999,9695,9726,9947,9598,9866,9721,...,9608,9668,9916,9790,9923,9914,9768,9794,161,154.17
2,-9167993136660569470,2,9251,9999,9605,9650,9896,9678,9857,9623,...,9549,9527,9895,9718,9930,9845,9697,9692,161,154.17
3,-9167993126042826314,2,9148,9998,9470,9455,9935,9531,9935,9419,...,9361,9372,9954,9402,9948,9926,9537,9671,161,154.17
4,-9167914043308884846,2,8973,9753,9347,9216,9783,9442,9966,9049,...,9312,8937,9910,9254,9868,9936,9409,9782,203,21.32


# feature_2_stand transform

In [225]:
# test_feautres.iloc[:, 2:258] = scaler.transform(test_feautres.iloc[:, 2:258])

In [283]:
test_feautres = prep.transform_with_scaler(test_feautres)

In [284]:
test_feautres = test_feautres.rename(columns = {i:f'feature_2_stand_{i}' for i in range(1, 1+ prep.features_num)})

In [285]:
test_feautres = test_feautres.drop(columns=[0])

In [286]:
test_feautres.head()

,id_job,feature_2_stand_1,feature_2_stand_2,feature_2_stand_3,feature_2_stand_4,feature_2_stand_5,feature_2_stand_6,feature_2_stand_7,feature_2_stand_8,feature_2_stand_9,...,feature_2_stand_249,feature_2_stand_250,feature_2_stand_251,feature_2_stand_252,feature_2_stand_253,feature_2_stand_254,feature_2_stand_255,feature_2_stand_256,max_feature_2_index,max_feature_2_abs_mean_diff
0,-9168029089769934451,0.768653,0.234160,0.719948,0.696445,-0.576554,0.802627,0.258123,0.706405,0.807730,...,0.829087,0.772043,0.120428,0.802338,-0.501696,0.184457,0.797741,0.730849,161,154.17
1,-9167993139315005259,-0.496477,0.234160,-0.162465,0.092886,0.583929,-0.398570,0.293919,0.030500,-0.419272,...,-0.189802,0.021290,0.438424,0.200994,0.419100,0.468539,-0.005891,-0.275585,161,154.17
2,-9167993136660569470,-0.501899,0.234160,-0.447115,-0.091333,0.419070,-0.131637,0.272442,-0.225248,-0.333114,...,-0.358190,-0.323519,0.378799,-0.028090,0.436243,0.305192,-0.277595,-0.890293,161,154.17
3,-9167993126042826314,-0.688054,0.232376,-0.874089,-0.563999,0.545139,-0.622126,0.458579,-0.757622,-0.814408,...,-0.894747,-0.702563,0.546315,-1.033512,0.480323,0.496947,-0.889886,-1.016850,161,154.17
4,-9167914043308884846,-1.004337,-0.204625,-1.263110,-1.143318,0.053792,-0.919089,0.532557,-1.723201,-1.574970,...,-1.034594,-1.766334,0.421388,-1.504407,0.284409,0.520621,-1.379719,-0.347903,203,21.32


# save

In [287]:
test_feautres.to_csv(PATH + '/data/processed/test_proc.tsv', sep ='\t', index = False)

In [288]:
pd.read_csv(PATH + '/data/processed/test_proc.tsv', sep ='\t')

,id_job,feature_2_stand_1,feature_2_stand_2,feature_2_stand_3,feature_2_stand_4,feature_2_stand_5,feature_2_stand_6,feature_2_stand_7,feature_2_stand_8,feature_2_stand_9,...,feature_2_stand_249,feature_2_stand_250,feature_2_stand_251,feature_2_stand_252,feature_2_stand_253,feature_2_stand_254,feature_2_stand_255,feature_2_stand_256,max_feature_2_index,max_feature_2_abs_mean_diff
0,-9168029089769934451,0.768653,0.234160,0.719948,0.696445,-0.576554,0.802627,0.258123,0.706405,0.807730,...,0.829087,0.772043,0.120428,0.802338,-0.501696,0.184457,0.797741,0.730849,161,154.17
1,-9167993139315005259,-0.496477,0.234160,-0.162465,0.092886,0.583929,-0.398570,0.293919,0.030500,-0.419272,...,-0.189802,0.021290,0.438424,0.200994,0.419100,0.468539,-0.005891,-0.275585,161,154.17
2,-9167993136660569470,-0.501899,0.234160,-0.447115,-0.091333,0.419070,-0.131637,0.272442,-0.225248,-0.333114,...,-0.358190,-0.323519,0.378799,-0.028090,0.436243,0.305192,-0.277595,-0.890293,161,154.17
3,-9167993126042826314,-0.688054,0.232376,-0.874089,-0.563999,0.545139,-0.622126,0.458579,-0.757622,-0.814408,...,-0.894747,-0.702563,0.546315,-1.033512,0.480323,0.496947,-0.889886,-1.016850,161,154.17
4,-9167914043308884846,-1.004337,-0.204625,-1.263110,-1.143318,0.053792,-0.919089,0.532557,-1.723201,-1.574970,...,-1.034594,-1.766334,0.421388,-1.504407,0.284409,0.520621,-1.379719,-0.347903,203,21.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-9163569547596238325,-0.077177,0.234160,-0.263674,0.075918,0.344721,0.095256,0.324942,-0.467948,-0.229131,...,0.141266,-0.113210,0.520762,-0.231720,0.507262,0.433029,-0.059467,-0.143000,161,154.17
96,-9163525656344858443,-0.044645,0.232376,-0.655858,-0.445227,0.661510,-0.542046,0.532557,-1.905878,-1.159038,...,-1.091675,-0.223255,0.475334,-0.578527,0.529302,0.475641,-0.824830,-1.034930,161,154.17
97,-9163472054053566485,-1.315198,0.218107,-2.031664,-1.986846,0.066722,-3.651813,0.379829,-3.205495,-4.400342,...,-4.773373,-2.519533,0.106232,-1.848032,0.152167,0.437764,-1.942261,-0.576912,161,154.17
98,-9163470125725648647,0.849983,0.235944,0.799018,0.754619,0.173396,0.926084,0.513466,0.755989,0.944393,...,0.923269,0.825843,0.512244,0.865973,0.441141,0.653192,0.874277,0.875487,1,172.12
